# Semantic RAG Traversal Algorithms Research
============================================================

## Table of Contents:
0. Introduction
1. Foundation - Understanding the Problem
2. Building Block 1 - Similarity Matrices
3. Building Block 2 - Distance Calculations
4. Building Block 3 - Dynamic Programming Framework
5. Building Block 4 - Advanced Optimization Tools
6. The Five Methods - Combining Building Blocks
7. Comparative Analysis and Validation

# Introduction/Overview

Semantic retrieval-augmented generation (RAG) systems for large language models (LLM's) invoke a pipeline of retrieving semantically relevant and accurate information to a user's query and feeding it into an LLM so that it provides a relevant and accurate answer. Research has been underway for several years regarding ways to enhance the performance, and accuracy of these systems, to maximize LLM effectiveness. This notebook details the research and discovery process of novel semantic retrieval augmented generation traversal algorithms, from beginning to end. 

My initial introduction to semantic RAG was by focusing on solving the chunking problem via the since-depreciated Bittensor "Chunking Subnet." This subnet focused on novel methods of semantic chunking, with a very specific reward heuristic, and it was by optimizing for this reward heuristic I discovered a way of visualizing and structuring documents based on semantic similarity by creating a similarity matrix of all pairwise sentence comparisons in a `numpy` array and architecting chunking algorithms from there.

As I learned more about the full semantic RAG pipeline I realized that *chunking might not even be an issue.* By sparsely connecting documents together based on semantic similarity, you can create a *knowledge graph* in which each chunk or sentence is connected to others that are similar. This is not necessarily new, as Microsoft discovered GraphRAG in its own research, but I experiemented with different knowledge graph architectures to see where opportunity might be. Specifically with respect to maximizing retrieval quality within semantic RAG systems as a whole.


During this process, I wondered if *knowledge graphs can be traversed.* As it turns out, *yes it's possible,* but precision and accuracy were yet to be determined. How does one algorithmically determine where to traverse to? Where to start and when to stop? And is it any better than basic semantic RAG with traditional blob vector storage? With this goal in mind, I discovered a custom knowledge graph architecture that is specifically designed to be traversed very rapidly, and designed *three basic traversal algorithms* that can be used to traverse a knowledge graph to retrieve contexts. These algorithms are the core of the completed research here and their strengths and weaknesses will be demonstrated in this notebook.

# 1.0: Identifying the Problem

When we use natural language processing to talk to LLM's, we need to ensure we give them semantically relevant, accurate information. We also want to make sure we don't give them *too* much information as well, otherwise they are likely to provide inaccurate information, or hallucinate. This is why we build RAG systems that allow us to retrieve relevant information so that a model can give an accurate output based on the user's input prompt.

Let's say a user talks to an LLM about a question inside a document. A typical LLM pipeline might look like this:

0. Prior to any user interaction, the document would be chunked according to semantic similarity, and these chunks of text would be stored in a "vector store" for retrieval upon prompting (explained later).
1. User inputs a prompt into an LLM.
2. User's prompt is first directly fed to an embedding model.
3. Embedding is sent to the vector store service that hosts the document's text and embeddings, almost like a "library lookup" for relevant text.
4. Most semantically relevant text from the document is returned back to the model as additional context, along with the original text prompt.
5. LLM is able to answer the question accurately and reliably without becoming overloaded.

Semantic RAG pipelines are typical engineered based on use-case. Retrieving medical papers or court documents is much different than retrieving Harry Potter novels. This being said, each step in the process has its own fine-tuning to be done, and like I said before, my original introduction into semantic RAG was through the chunking problem.

# 2.0: Solving Semantic RAG Chunking Through Bittensor

The beginning of this research started via *semantic RAG chunking.*

How would you separate a Harry Potter article into paragraphs, without having the indentations? What about a phone book, or an excel spreadsheet full of medical records? What about a long series of emails across multiple topics? This is the core problem of chunking; which is separating out chunks of documents based on semantic similarity. It's an arguably infinitely complex problem. Not because humans can't do it, but because there's technically no right or wrong answer. Chunking heavily depends on the use-case of the documents being chunked. And how does one measure "accurate" chunking to begin with anyway?

Enter Bittensor; a network of engineers being incentivized to solve impossible problems. From 2024-2025, the subnet known as Chunking, set out to solve this exact problem, with its own reward structure and heuristic for doing so. Miners were tasked with chunking documents based semantic similarity, using their own custom algorithms, and competed to return the "best" possible chunked documents.

The next section comes directly from the evaluation documentation from Chunking and it outlines the reward heuristic in detail.

## Reward Function

This subnet incentivizes responses that maximize intrachunk similarity and interchunk dissimilarity, without exceeding variable constraints, such as time or maximum chunk length. This subnet measures similarity by the dot product.

Since the similarity score can be influenced significantly by the content in the provided document, evaluation is always done relatively, within groups of miners that all faced the same query. See [Incentive Mechanism](./incentive_mechanism.md) to learn more.

In the default validator, [reward.py](../chunking/validator/reward.py) scores the responses of each miner.

## Failure States

### 1. No new words

First, the validator confirms that each word in the chunked response also exists, in the same order, in the original document.

```python
# check that every word in chunk exists and is in the same order as the source document
chunk_words = ' '.join(word_tokenize(chunks[i]))
combined_chunk_words += ' ' + chunk_words
if chunk_words not in ' '.join(document_words):
    return 0
```

### 2. All words present

Then, the validator confirms that every set of 3 adjacent words in the original document is also present within the chunked response.

```python
# check that every set of 3 adjacent words from the document appears in the chunks
for i in range(0, len(document_words), 3):
    if (len(' '.join(document_words[i:i+3])) < chunk_size
        and ' '.join(document_words[i:i+3]) not in combined_chunk_words):
        return 0
```

If either don't hold true, the score is 0.

## Evaluating

After passing the fail states, the validator parses through each chunk, creating 'small chunks' of 3 sentences or fewer.

```python
# create test segments
sentences = sent_tokenize(chunks[i])
for j in range(0, len(sentences), 3):
    text = " ".join(sentences[j:j+3])
    smallChunks.append(smallChunk(i, text))
```

A random sample, of `num_embeddings` size, is taken and then embedded. The default value is 150.

```python
# pick out segments to use for evaluation
if num_embeddings < len(smallChunks):
    testChunks = sample(smallChunks, num_embeddings)
else:
    testChunks = smallChunks
```

Then, to calculate the similarity score, the dot product of every possible pair of embeddings is calculated. The average of each pair originating from the same chunk is added to the score (intrachunk similarity), while the average of each pair originating from different chunks is subtracted from the score (interchunk dissimilarity).

```python
for i in range(len(testChunks) - 1):
    j = i + 1
    while j < len(testChunks):
        if testChunks[i].sourceChunk == testChunks[j].sourceChunk:
            reward += np.dot(np.asarray(embeddings[i]), np.asarray(embeddings[j]))
        else:
            reward -= np.dot(np.asarray(embeddings[i]), np.asarray(embeddings[j]))
        j += 1

reward = (
    (np.mean(intrachunk_similarities) if len(intrachunk_similarities) > 0 else 0)
    - (np.mean(interchunk_similarities) if len(interchunk_similarities) > 0 else 0)
)
```

Here is a visualization of how the validator calculates a miner’s score:

![evaluations](docs/evaluations.png)

## Penalties

Finally, penalities are deducted from the score.

Responses are penalized exponentially for each character over the maximum chunk length: `chunk_size`

```python
# add up size penalty to be applied later
chunk_length = len(chunks[i])
if chunk_length > chunk_size:
    size_penalty += ((chunk_length / chunk_size) - 1) * 10
    _verbose(f"Chunk {i} is too long: {chunk_length} characters, new size penalty: {size_penalty}")

```

And for each chunk over the maximum chunk quantity: `chunk_qty`

```python
# penalize an excessive number of chunks
    num_chunks = len(chunks)
    if num_chunks > chunk_qty:
        qty_penalty += 10 * ((num_chunks / chunk_qty) - 1) * 10
        _verbose(f"Too many chunks: {num_chunks} chunks, new quantity penalty: {qty_penalty}")
```

```python
reward *= (2/3) ** (size_penalty + qty_penalty)
```

Finally, note that there is a soft-time limit (default: 3.75 seconds). Validators exponentially penalize responses for each second they are late.

```python
if response.dendrite.process_time > response.time_soft_max:
    over_time = response.dendrite.process_time - response.time_soft_max
    _verbose(f"Applying time penalty: {over_time} seconds over time")
    reward *= (2/3) ** over_time
```

This method of scoring chunks was applied to fully chunked documents, typically resulted in a score near `1.0`, using the `text-embedding-ada-002` model from OpenAI. Crucially, the subnet required all chunks in the document to be within a max chunk size and chunk quantity. By maximizing the difference between intrachunk and interchunk similarity across many random samples, while staying within a max chunk size and max chunk quantity, algorithms highly optimized for this heuristic gradually produced better scores over time.

# 3.0: The Similarity Matrix

The most important takeaway was that this strategy of scoring required outside-the-box thinking in terms of *global optimization*. After many failed attempts with different algorithmic approaches to chunking, I found that the best way to tackle this problem was by *comparing every single sentence to every other sentence* as a starting point. It looks like this:

![SIMILARITYMATRIX](docs/sample.png)

This is the *Neuroscience* article on Wikipedia. The document reads from top left to bottom right. You can see areas with high dot product/cosine similarity are in red, and low similarity are in blue. When chunking, this allowed me to "visualize" the chunks. You can see how certain sections, like sentences ~3 to ~15 share very little similarity with the rest of the document, as evidenced by the low similarity when compared to the rest of the document.

It should also be mentioned that while the subnet used dot-product, we will use cosine similarity to keep things simple.

Let's build a full similarity matrix right now using assets from the repository:

### First let's choose some topics for the remainder of this notebook. You may change these if you wish:

In [ ]:
# Using 3 related topics for cross-document visualization
topics = ["Machine Learning", "Psychology", "Deep Learning"]

### Next let's import our modules:

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd() 
sys.path.append(str(project_root))

from utils.wiki import WikiEngine
from utils.chunking import ChunkEngine
from utils.models import MultiGranularityEmbeddingEngine
from utils.similarity import SimilarityEngine
from utils.knowledge_graph import KnowledgeGraphBuilder

logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Create temp directory in the project in a folder called "temp_data"
Path('temp_data').mkdir(exist_ok=True)

print("Imports successful.")

### Now we'll set configuration settings:

In [ ]:
import torch
import os

# ============================================================================
# API KEYS - Set your API keys here
# ============================================================================
# For Google Colab or local use, set your OpenAI API key here
# You can also use environment variables by setting OPENAI_API_KEY in your .env file

OPENAI_API_KEY = "optional-api-key"

# Set the environment variable so the code can access it
if OPENAI_API_KEY != "your-openai-api-key-here":
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print("OpenAI API key set")
else:
    # Try to load from environment
    if 'OPENAI_API_KEY' in os.environ:
        print("Using OpenAI API key from environment")
    else:
        print("WARNING: No OpenAI API key found.")

# ============================================================================
# CONFIGURATION
# ============================================================================

# Auto-detect best device for your system
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'  # Apple Silicon GPU
else:
    device = 'cpu'

print(f"Using device: {device}")
print(f"Loading {len(topics)} related topics: {topics}")

config = {
    'directories': {'data': 'temp_data', 'embeddings': 'temp_data/embeddings'},
    'system': {'device': device},  # Auto-detected device
    'wikipedia': {
        'topics': topics,  # 3 related documents
        'articles_per_topic': 1,
        'max_article_length': 500000,
        'min_article_length': 1000,
        'language': 'en',
        'rate_limit_delay': 1.0,
        'retry_attempts': 3,
        'timeout_seconds': 30,
        'use_cached_articles': False
    },
    'text_processing': {
        'clean_html': True, 'remove_references': True, 'remove_navigation': True,
        'remove_tables': True, 'fix_encoding': True, 'normalize_whitespace': True,
        'min_sentence_length': 10, 'max_sentence_length': 500
    },
    'chunking': {
        'strategy': 'sliding_window', 'window_size': 3, 'overlap': 2
    },
    'models': {
        'embedding_models': ['mixedbread-ai/mxbai-embed-large-v1'],
        'embedding_batch_size': 32,
        'granularity_types': {
            'chunks': {'enabled': True},
            'sentences': {'enabled': True},
            'doc_summaries': {'enabled': True}  
        }
    },
    'similarities': {
        'similarity_metric': 'cosine', 'batch_size': 500,
        'granularity_types': {
            'chunk_to_chunk': {
                'enabled': True,
                'intra_document': {'enabled': True, 'top_k': 10, 'min_threshold': 0.3},
                'inter_document': {'enabled': True, 'top_x': 5, 'min_threshold': 0.3}
            }
        }
    },
    'theme_extraction': {
        'num_themes': 5,  # Number of themes to extract per document
        'use_openai': False,  # Use OpenAI instead of Ollama
        'openai_model': 'gpt-4o-mini',  # Cheaper model for theme extraction
        'temperature': 0.1
    },
    'knowledge_graph': {
        'quality_scoring': {
            'enabled': False  # Keep this disabled - only need themes, not quality scoring
        }
    },
    'retrieval': {
        'semantic_traversal': {
            'max_hops': 10,
            'max_sentences': 20,
            'similarity_threshold': 0.3,
            'early_stopping': {
                'enabled': True,
                'min_sentences': 5,
                'similarity_drop_threshold': 0.15,
                'consecutive_drops': 3
            }
        }
    },
    'reranking': {
        'enabled': True,
        'top_k': 20
    },
    'deepeval': {
        'models': {
            'question_generation': {
                'provider': 'openai',
                'model_name': 'gpt-4o',
                'temperature': 0.7
            },
            'answer_generation': {
                'provider': 'openai',
                'model_name': 'gpt-4o',
                'temperature': 0.7
            },
            'evaluation_judge': {
                'provider': 'openrouter',
                'model_name': 'meta-llama/llama-3.3-70b-instruct',
                'temperature': 0.0
            }
        }
    }
}

print(f"✅ Configuration set for {len(topics)} topics")
print(f"🎨 Theme extraction: {'OpenAI' if config['theme_extraction']['use_openai'] else 'Ollama'}")

### Let's fetch 3 Wikipedia documents on our topics:

In [ ]:
print(f"🔍 Fetching Wikipedia articles for {len(topics)} topics: {topics}")
wiki_engine = WikiEngine(config, logger)
articles = wiki_engine.acquire_articles("temp_data/wiki_multi.json")

if not articles:
    raise ValueError(f"Could not fetch articles for topics: {topics}")

print(f"✅ Fetched {len(articles)} articles:")
total_sentences = 0
for i, article in enumerate(articles):
    print(f"   {i+1}. {article.title} ({len(article.sentences)} sentences)")
    total_sentences += len(article.sentences)
    print(f"      Preview: {article.cleaned_text[:150]}...")
    print()

print(f"📄 Total corpus: {total_sentences} sentences across {len(articles)} documents")

### Now let's chunk our documents:

We're going to use *three sentence sliding windows* with a 2 sentence overlap. This would look like:

Chunk 1. [S1, S2, S3]

Chunk 2. [S2, S3, S4]

Chunk 3. [S3, S4, S5]

In [ ]:
chunk_engine = ChunkEngine(config, logger)
chunks = chunk_engine.create_chunks(articles)

# Show document breakdown - FIX: use source_article instead of source_document
chunks_per_doc = {}
for chunk in chunks:
    doc_name = chunk.get('source_article', 'Unknown')  # Fixed: changed from source_document
    chunks_per_doc[doc_name] = chunks_per_doc.get(doc_name, 0) + 1

print("\n📊 Chunks per document:")
for doc_name, count in chunks_per_doc.items():
    print(f"   {doc_name}: {count} chunks")

### Now let's embed each 3 sentence sliding window chunk:

In [ ]:
embedding_engine = MultiGranularityEmbeddingEngine(config, logger)
embeddings = embedding_engine.generate_embeddings(chunks, articles)
print(f"🧠 Generated embeddings")

# Show embedding info
model_name = list(embeddings.keys())[0]
chunk_embeddings = embeddings[model_name]['chunks']
print(f"📊 Embedding info: {len(chunk_embeddings)} chunks")

### Now let's build similarity matrices from our documents directly from embeddings

In [ ]:
# Group embeddings by document for clarity
doc_embeddings = {}
doc_names = []

for chunk_emb in chunk_embeddings:
    doc_name = chunk_emb.source_article if hasattr(chunk_emb, 'source_article') else 'Unknown'
    
    if doc_name not in doc_embeddings:
        doc_embeddings[doc_name] = []
        doc_names.append(doc_name)
    
    doc_embeddings[doc_name].append(chunk_emb.embedding)

print(f"\n📚 Available documents:")
for i, (doc_name, embeddings_list) in enumerate(doc_embeddings.items()):
    print(f"   {i}: '{doc_name}' ({len(embeddings_list)} chunks)")

# Store for next step
globals()['doc_embeddings'] = doc_embeddings
globals()['doc_names'] = doc_names

In [ ]:
# DOCUMENT SELECTION - Change this to try different documents

# CHOOSE WHICH DOCUMENT TO VISUALIZE:

chosen_doc = "Machine learning"
# chosen_doc = "Neural network (machine learning)" 
# chosen_doc = "Deep learning"

print(f"\n✅ Ready to compute similarity matrix for: '{chosen_doc}'")

# Store for next step
globals()['chosen_doc'] = chosen_doc

In [ ]:
# Import cosine_similarity for computing the full matrix
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity matrix for the chosen document
print(f"🔄 Computing similarity matrix for: '{chosen_doc}'")

# Get embeddings for chosen document
chosen_embeddings = doc_embeddings[chosen_doc]

# Convert to numpy matrix
embedding_matrix = np.array(chosen_embeddings)
print(f"\n🧮 Embedding matrix shape: {embedding_matrix.shape}")
print(f"   → {embedding_matrix.shape[0]} chunks × {embedding_matrix.shape[1]} dimensions")

# Compute the FULL similarity matrix (no filtering!)
print("\n🔄 Computing cosine similarity matrix:")
print(f"   Total comparisons: {embedding_matrix.shape[0] * (embedding_matrix.shape[0] - 1) // 2}")

similarity_matrix = cosine_similarity(embedding_matrix)

print(f"\n✅ Similarity matrix created!")
print(f"   📊 Shape: {similarity_matrix.shape}")
print(f"   📊 Value range: {similarity_matrix.min():.6f} to {similarity_matrix.max():.6f}")
print(f"   📊 Diagonal values: {np.diag(similarity_matrix)[:5]} (should be 1.0)")
print(f"   📊 Total elements: {similarity_matrix.size:,}")
print(f"   📊 Zeros: {np.sum(similarity_matrix == 0.0)} (should be 0 for unfiltered matrix)")

# Store for visualization step
globals()['similarity_matrix'] = similarity_matrix

### Visualize the Similarity Matrix

Now let's see what this similarity matrix actually looks like! This visualization shows the semantic landscape of our chosen document.

In [ ]:
# Create the visualization using seaborn heatmap
fig, ax = plt.subplots(figsize=(12, 8), dpi=150)

# Create heatmap
sns.heatmap(
    similarity_matrix,
    cmap='RdYlBu_r',  # Red (high similarity) to Blue (low similarity)
    vmin=0.0,
    vmax=1.0,
    square=True,
    cbar_kws={'label': 'Cosine Similarity'},
    ax=ax
)

ax.set_title(f"{chosen_doc} - Semantic Similarity Matrix ({similarity_matrix.shape[0]} chunks)", 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Chunk Index', fontsize=12)
ax.set_ylabel('Chunk Index', fontsize=12)

plt.tight_layout()
plt.show()

print(f"🎨 Visualizing similarity matrix for: '{chosen_doc}'")
print(f"📊 This {similarity_matrix.shape[0]}×{similarity_matrix.shape[0]} matrix contains {similarity_matrix.size:,} similarity scores")

# 4.0: The Knowledge Graph

Using document similarity matrices, then building off Microsoft's GraphRAG, it is possible to create a *knowledge graph* that is, in essence, *many similarity matrices from different documents connected together.* In order to keep the file size down, we can sparse out our similarity matrices significantly by only storing the *most similar sentences* to each sentence, rather than every single pairwise comparison. This also includes *between-document connections*. 

We use `top_k` as our variable for top intra-document similarities, and `top_x` as our variable for top inter-document similarities. What then happens is each chunk gets connected to `top_k` number of chunks within the same document, and `top_x` number of chunks in all other documents we choose to store in our knowledge graph. This drastically minimizes the number of connections in our graph significantly.


Now, we've only been working with 3 sentence sliding window chunks, so why not use *individual sentences as well*? We can do this by including, in our knowledge graph, *individual sentences* and then *only storing them as children to the parent windows they came from.* This allows us to reference sentences within our knowledge graph for sentence-level accuracy, while minimizing unnecessary noise, as sentence nodes in the graph will only connect to their respective parent chunks.

Finally, we want to add *hierarchical properties* to each node. RAGAS and other libraries like it use tools like NER (Named Entity Recognition) to extract named entities and themes. To keep things simple, we can simply take the first 500 words of a Wikipedia document, give the whole thing to an LLM of choice, then have it generate *5 unique themes* from the summary. These 5 themes then get passed down to *all chunk and sentence nodes* inside that particular document. We can also take those five themes as a list, embed them as one chunk, then compare all thematic chunk similarities between each other to then, create another bridge between the most similar documents by theme, denoted by a `top_t`.

The final result is a *lightweight knowledge graph* that is designed to be traversed. 

In [ ]:
# Extract themes from document summaries using OpenAI
from utils.extraction import ThemeExtractionEngine

print("🎨 Extracting themes from document summaries...")

theme_engine = ThemeExtractionEngine(config, logger)
theme_data = theme_engine.extract_themes(
    multi_granularity_embeddings=embeddings,
    articles=articles,
    force_recompute=True  # FORCE FRESH EXTRACTION with OpenAI
)

# Display extracted themes
print(f"\n📊 Extracted Themes:")
for theme_result in theme_data['extraction_results']['document_themes']:
    print(f"\n   {theme_result.doc_title}:")
    for i, theme in enumerate(theme_result.themes, 1):
        print(f"      {i}. {theme}")
    print(f"   Method: {theme_result.extraction_method} | Model: {theme_result.model_used}")

print(f"\n✅ Theme extraction complete!")

In [ ]:
# Build similarities and knowledge graph
print("🔗 Computing similarity matrices...")

similarity_engine = SimilarityEngine(config, logger)
similarities = similarity_engine.compute_similarity_matrices(embeddings)

print(f"\n🏗️ Building knowledge graph...")

kg_builder = KnowledgeGraphBuilder(config, logger)
knowledge_graph = kg_builder.build_knowledge_graph(
    chunks=chunks,
    multi_granularity_embeddings=embeddings,
    similarity_data=similarities,
    theme_data=theme_data
)

print(f"\n✅ Knowledge graph built!")
print(f"   📊 Total chunks: {len(knowledge_graph.chunks)}")
print(f"   📊 Total documents: {len(knowledge_graph.documents)}")
print(f"   📊 Total sentences: {len(knowledge_graph.sentences)}")

# Initialize retrieval orchestrator
print(f"🔍 Initializing retrieval orchestrator...")
from utils.retrieval import RetrievalOrchestrator

retrieval_orchestrator = RetrievalOrchestrator(
    knowledge_graph=knowledge_graph,
    config=config,
    logger=logger
)

print(f"✅ Retrieval orchestrator ready with {len(retrieval_orchestrator.algorithms)} algorithms")

In [ ]:
import plotly.graph_objects as go
from collections import defaultdict
import numpy as np
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity as compute_cosine_similarity

print(f"\n🎨 Creating Stage 3: Adding document themes at top...")

# Reuse the same document grouping
doc_chunks_viz3 = defaultdict(list)
chunk_to_doc_idx_viz3 = {}

for chunk_id, chunk in knowledge_graph.chunks.items():
    doc_name = chunk.source_document
    doc_chunks_viz3[doc_name].append(chunk)

doc_names_sorted_viz3 = sorted(doc_chunks_viz3.keys())[:2]

for doc_name in doc_names_sorted_viz3:
    doc_chunks_viz3[doc_name].sort(key=lambda c: c.chunk_id)
    for idx, chunk in enumerate(doc_chunks_viz3[doc_name]):
        chunk_to_doc_idx_viz3[chunk.chunk_id] = (doc_name, idx)

# Get theme data and embeddings
doc_themes = {}  # doc_name -> list of themes
doc_theme_text = {}  # doc_name -> full theme text for embedding
doc_theme_embeddings = {}  # doc_name -> embedding vector

for theme_result in theme_data['extraction_results']['document_themes']:
    doc_title = theme_result.doc_title
    themes = theme_result.themes
    
    # The text that was embedded is the list of themes joined
    theme_text = ", ".join(themes)
    
    doc_themes[doc_title] = themes
    doc_theme_text[doc_title] = theme_text
    
print(f"\n   📋 Document themes:")
for doc_name in doc_names_sorted_viz3:
    if doc_name in doc_themes:
        print(f"      {doc_name}: {doc_themes[doc_name]}")

# Get theme similarity from knowledge graph (uses fresh OpenAI theme embeddings)
# Map document names to document IDs from knowledge graph
doc_name_to_id = {}
for doc_id, doc in knowledge_graph.documents.items():
    doc_name_to_id[doc.title] = doc_id

# Get theme similarity between the two visualized documents
if len(doc_names_sorted_viz3) == 2:
    doc_id_1 = doc_name_to_id.get(doc_names_sorted_viz3[0])
    doc_id_2 = doc_name_to_id.get(doc_names_sorted_viz3[1])
    
    print(f"\n   🔍 Debug: doc_names_sorted_viz3 = {doc_names_sorted_viz3}")
    print(f"   🔍 Debug: doc_id_1 = {doc_id_1}, doc_id_2 = {doc_id_2}")
    
    if doc_id_1 and doc_id_2:
        # Get similarity from knowledge graph (check both directions since top_r=1)
        theme_similarity = knowledge_graph.get_theme_similarity_score(doc_id_1, doc_id_2)
        if theme_similarity == 0.0:
            theme_similarity = knowledge_graph.get_theme_similarity_score(doc_id_2, doc_id_1)
        print(f"   🔗 Theme similarity from knowledge graph: {theme_similarity:.4f}")
    else:
        theme_similarity = 0.0
        print(f"   ⚠️  Could not find document IDs in knowledge graph")
else:
    theme_similarity = 0.0
    print(f"   ⚠️  Need exactly 2 documents for theme similarity")

# Create figure
fig3 = go.Figure()

# Layout parameters - wider spacing, reasonable column width
column_spacing = 10.0  # INCREASED: More space between columns to avoid overlap
column_width = 6.5     # Reasonable width for each column
top_section_height = 0.15
middle_section_height = 0.55
bottom_section_height = 0.18

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
doc_colors = {doc: colors[i] for i, doc in enumerate(doc_names_sorted_viz3)}

max_chunks = max(len(doc_chunks_viz3[doc]) for doc in doc_names_sorted_viz3)

# Calculate section boundaries
middle_y_start = top_section_height * max_chunks
middle_y_end = (top_section_height + middle_section_height) * max_chunks
sentence_y_start = middle_y_end
sentence_y_end = (top_section_height + middle_section_height + bottom_section_height) * max_chunks

# Reserve space for "Knowledge Graph" title at top of middle section
kg_label_space = 3  # Space for the label
kg_graph_y_start = middle_y_start + kg_label_space
kg_graph_y_end = middle_y_end

chunk_positions_viz3 = {}

# === STAGE 1 CONTENT: Middle section with 2D graph ===
for doc_idx, doc_name in enumerate(doc_names_sorted_viz3):
    chunks_in_doc = doc_chunks_viz3[doc_name]
    n_chunks = len(chunks_in_doc)
    x_center = doc_idx * column_spacing
    
    # Build distance matrix and layout
    distance_matrix = np.ones((n_chunks, n_chunks)) * 2.0
    for i, chunk in enumerate(chunks_in_doc):
        for target_id in chunk.intra_doc_connections:
            if target_id in chunk_to_doc_idx_viz3:
                tgt_doc, tgt_idx = chunk_to_doc_idx_viz3[target_id]
                if tgt_doc == doc_name:
                    similarity = chunk.connection_scores.get(target_id, 0.0)
                    distance = 1.0 - similarity
                    distance_matrix[i, tgt_idx] = distance
                    distance_matrix[tgt_idx, i] = distance
    
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    positions_2d = mds.fit_transform(distance_matrix)
    
    x_min, x_max = positions_2d[:, 0].min(), positions_2d[:, 0].max()
    y_min, y_max = positions_2d[:, 1].min(), positions_2d[:, 1].max()
    x_range = x_max - x_min if x_max != x_min else 1.0
    y_range = y_max - y_min if y_max != y_min else 1.0
    
    # Center horizontally: normalize to 0-1, then scale around x_center
    x_normalized = (positions_2d[:, 0] - x_min) / x_range
    # Scale to fit within column width, centered at x_center
    x_scaled = x_center + (x_normalized - 0.5) * column_width * 0.75  # 75% of width for padding
    
    # Center vertically: normalize to 0-1, then scale around vertical center
    y_normalized = (positions_2d[:, 1] - y_min) / y_range
    graph_height = kg_graph_y_end - kg_graph_y_start
    graph_center_y = kg_graph_y_start + graph_height / 2
    # Scale to fit within available height, centered vertically
    y_scaled = graph_center_y + (y_normalized - 0.5) * graph_height * 0.75  # 75% of height for padding
    
    for i, chunk in enumerate(chunks_in_doc):
        chunk_positions_viz3[chunk.chunk_id] = (x_scaled[i], y_scaled[i])
    
    # Draw edges (intra-doc)
    edge_x, edge_y = [], []
    for chunk in chunks_in_doc:
        src_pos = chunk_positions_viz3[chunk.chunk_id]
        for target_id in chunk.intra_doc_connections:
            if target_id in chunk_positions_viz3 and target_id in chunk_to_doc_idx_viz3:
                tgt_doc, _ = chunk_to_doc_idx_viz3[target_id]
                if tgt_doc == doc_name:
                    tgt_pos = chunk_positions_viz3[target_id]
                    edge_x.extend([src_pos[0], tgt_pos[0], None])
                    edge_y.extend([src_pos[1], tgt_pos[1], None])
    
    if edge_x:
        fig3.add_trace(go.Scatter(
            x=edge_x, y=edge_y,
            mode='lines',
            line=dict(color='#CCCCCC', width=0.5),
            hoverinfo='skip',
            showlegend=False
        ))
    
    # Draw chunk nodes
    node_x = [chunk_positions_viz3[c.chunk_id][0] for c in chunks_in_doc]
    node_y = [chunk_positions_viz3[c.chunk_id][1] for c in chunks_in_doc]
    node_text = [f"Chunk {i}<br>{c.chunk_text[:80]}..." for i, c in enumerate(chunks_in_doc)]
    
    fig3.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker=dict(size=6, color=doc_colors[doc_name], line=dict(width=1, color='black')),
        text=node_text,
        hovertemplate='%{text}<extra></extra>',
        name=doc_name,
        showlegend=True
    ))

# Inter-document connections
inter_edge_x, inter_edge_y = [], []
for chunk_id, chunk in knowledge_graph.chunks.items():
    if chunk_id not in chunk_positions_viz3:
        continue
    src_pos = chunk_positions_viz3[chunk_id]
    for target_id in chunk.inter_doc_connections:
        if target_id in chunk_positions_viz3:
            tgt_pos = chunk_positions_viz3[target_id]
            inter_edge_x.extend([src_pos[0], tgt_pos[0], None])
            inter_edge_y.extend([src_pos[1], tgt_pos[1], None])

if inter_edge_x:
    fig3.add_trace(go.Scatter(
        x=inter_edge_x, y=inter_edge_y,
        mode='lines',
        line=dict(color='red', width=0.5, dash='dot'),
        hoverinfo='skip',
        showlegend=False,
        opacity=0.5
    ))

# === STAGE 2 CONTENT: Bottom section with chunks 5, 6, 7 and sentences ===
print(f"   📊 Adding sentence layer for chunks 5, 6, 7...")

sentence_count = 0
for doc_idx, doc_name in enumerate(doc_names_sorted_viz3):
    chunks_in_doc = doc_chunks_viz3[doc_name]
    x_center = doc_idx * column_spacing
    
    if len(chunks_in_doc) > 7:
        chunks_to_show = chunks_in_doc[5:8]
    else:
        chunks_to_show = chunks_in_doc[-3:] if len(chunks_in_doc) >= 3 else chunks_in_doc
    
    chunk_rect_width = column_width * 0.25
    chunk_rect_height = (sentence_y_end - sentence_y_start) * 0.15
    chunk_spacing = column_width * 0.03
    
    total_chunks_width = 3 * chunk_rect_width + 2 * chunk_spacing
    start_x = x_center - total_chunks_width / 2
    
    all_chunk_sentences = []
    for chunk in chunks_to_show:
        sentences = knowledge_graph.get_chunk_sentences(chunk.chunk_id) or []
        all_chunk_sentences.append(sentences)
    
    sentence_to_chunks = {}
    all_unique_sentence_ids = []
    sentence_obj_map = {}
    
    for chunk_idx, sentences in enumerate(all_chunk_sentences):
        for s in sentences[:3]:
            if s.sentence_id not in all_unique_sentence_ids:
                all_unique_sentence_ids.append(s.sentence_id)
                sentence_obj_map[s.sentence_id] = s
            if s.sentence_id not in sentence_to_chunks:
                sentence_to_chunks[s.sentence_id] = []
            sentence_to_chunks[s.sentence_id].append(chunk_idx)
    
    chunk_centers = []
    
    for chunk_idx, chunk in enumerate(chunks_to_show):
        chunk_global_idx = chunks_in_doc.index(chunk)
        chunk_x = start_x + chunk_idx * (chunk_rect_width + chunk_spacing)
        chunk_y = sentence_y_start + (sentence_y_end - sentence_y_start) * 0.15
        
        chunk_centers.append((chunk_x + chunk_rect_width/2, chunk_y + chunk_rect_height))
        
        fig3.add_shape(
            type="rect",
            x0=chunk_x, x1=chunk_x + chunk_rect_width,
            y0=chunk_y, y1=chunk_y + chunk_rect_height,
            line=dict(color=doc_colors[doc_name], width=2),
            fillcolor=f'rgba{tuple(list(int(doc_colors[doc_name].lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + [0.2])}',
            layer='below'
        )
        
        fig3.add_annotation(
            x=chunk_x + chunk_rect_width/2,
            y=chunk_y + chunk_rect_height/2,
            text=f"<b>C{chunk_global_idx}</b>",
            showarrow=False,
            font=dict(size=9, color='black')
        )
        
        fig3.add_trace(go.Scatter(
            x=[chunk_x + chunk_rect_width/2],
            y=[chunk_y + chunk_rect_height/2],
            mode='markers',
            marker=dict(size=20, color='rgba(0,0,0,0)'),
            text=[f"<b>Chunk {chunk_global_idx}</b><br>{chunk.chunk_text[:100]}..."],
            hovertemplate='%{text}<extra></extra>',
            showlegend=False
        ))
    
    if all_unique_sentence_ids:
        sentence_y_pos = chunk_y + chunk_rect_height + (sentence_y_end - sentence_y_start) * 0.25
        total_span_width = total_chunks_width
        sentence_spacing_val = total_span_width / (len(all_unique_sentence_ids) + 1)
        
        sent_x_positions = []
        sent_y_positions = []
        sent_texts = []
        
        for sent_idx, sent_id in enumerate(all_unique_sentence_ids):
            sent_x = start_x + (sent_idx + 1) * sentence_spacing_val
            sentence_obj = sentence_obj_map[sent_id]
            
            sent_x_positions.append(sent_x)
            sent_y_positions.append(sentence_y_pos)
            
            chunk_indices = sentence_to_chunks[sent_id]
            chunk_labels = [str(chunks_in_doc.index(chunks_to_show[i])) for i in chunk_indices if i < len(chunks_to_show)]
            sent_texts.append(f"<b>S (chunks: {','.join(chunk_labels)})</b><br>{sentence_obj.sentence_text[:100]}...")
            
            for chunk_idx in chunk_indices:
                if chunk_idx < len(chunk_centers):
                    chunk_cx, chunk_cy = chunk_centers[chunk_idx]
                    fig3.add_trace(go.Scatter(
                        x=[chunk_cx, sent_x],
                        y=[chunk_cy, sentence_y_pos],
                        mode='lines',
                        line=dict(color='#666666', width=1),
                        hoverinfo='skip',
                        showlegend=False
                    ))
        
        fig3.add_trace(go.Scatter(
            x=sent_x_positions,
            y=sent_y_positions,
            mode='markers',
            marker=dict(size=8, color='lightgray', line=dict(color='black', width=1.5)),
            text=sent_texts,
            hovertemplate='%{text}<extra></extra>',
            showlegend=False,
            name=f'Sentences'
        ))
        
        sentence_count += len(all_unique_sentence_ids)

# === STAGE 3 CONTENT: Top section with document themes ===
print(f"   🎨 Adding document themes at top...")

theme_node_positions = {}  # Store theme rectangle edges for connection line
theme_rect_bounds = {}  # Store x0, x1 for each doc

for doc_idx, doc_name in enumerate(doc_names_sorted_viz3):
    x_center = doc_idx * column_spacing
    
    if doc_name in doc_themes:
        # Theme rectangle
        theme_rect_width = column_width * 0.8
        theme_rect_height = middle_y_start * 0.6
        theme_x = x_center - theme_rect_width / 2
        theme_y = middle_y_start * 0.2
        
        # Store edge positions for connection line
        theme_node_positions[doc_name] = (x_center, theme_y + theme_rect_height / 2)
        theme_rect_bounds[doc_name] = {
            'x0': theme_x,
            'x1': theme_x + theme_rect_width,
            'y': theme_y + theme_rect_height / 2
        }
        
        # Draw theme rectangle
        fig3.add_shape(
            type="rect",
            x0=theme_x, x1=theme_x + theme_rect_width,
            y0=theme_y, y1=theme_y + theme_rect_height,
            line=dict(color=doc_colors[doc_name], width=3),
            fillcolor=f'rgba{tuple(list(int(doc_colors[doc_name].lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + [0.3])}',
            layer='below'
        )
        
        # Wrap theme text for better display - insert line breaks intelligently
        theme_text_raw = doc_theme_text[doc_name]
        # Split by commas and rejoin with line breaks every 2-3 items
        theme_parts = [t.strip() for t in theme_text_raw.split(',')]
        wrapped_lines = []
        current_line = []
        current_length = 0
        
        for part in theme_parts:
            # If adding this part would make line too long (>40 chars), start new line
            if current_length + len(part) > 40 and current_line:
                wrapped_lines.append(', '.join(current_line))
                current_line = [part]
                current_length = len(part)
            else:
                current_line.append(part)
                current_length += len(part) + 2  # +2 for ", "
        
        if current_line:
            wrapped_lines.append(', '.join(current_line))
        
        theme_text_wrapped = '<br>'.join(wrapped_lines)
        
        # Add theme text with wrapping
        fig3.add_annotation(
            x=theme_x + theme_rect_width/2,
            y=theme_y + theme_rect_height/2,
            text=f"<b>THEMES:</b><br>{theme_text_wrapped}",
            showarrow=False,
            font=dict(size=9, color='black'),
            align='center'
        )
        
        # Add hover area to show full theme list
        fig3.add_trace(go.Scatter(
            x=[x_center],
            y=[theme_y + theme_rect_height/2],
            mode='markers',
            marker=dict(size=30, color='rgba(0,0,0,0)'),
            text=[f"<b>Document Themes (Embedded Text):</b><br>{theme_text_raw}<br><br>Similarity: {theme_similarity:.4f}"],
            hovertemplate='%{text}<extra></extra>',
            showlegend=False
        ))

# Draw theme similarity connection line at the edges
if len(theme_rect_bounds) == 2:
    doc_list = list(theme_rect_bounds.keys())
    bounds1 = theme_rect_bounds[doc_list[0]]
    bounds2 = theme_rect_bounds[doc_list[1]]
    
    # Connect from right edge of left rect to left edge of right rect
    x1 = bounds1['x1']  # Right edge of first theme
    y1 = bounds1['y']
    x2 = bounds2['x0']  # Left edge of second theme
    y2 = bounds2['y']
    
    fig3.add_trace(go.Scatter(
        x=[x1, x2],
        y=[y1, y2],
        mode='lines',
        line=dict(color='red', width=0.5),
        hoverinfo='skip',
        showlegend=False,
        name='Theme Similarity'
    ))

print(f"   ✅ Document themes added with similarity: {theme_similarity:.4f}")

# Draw column boundaries
for doc_idx, doc_name in enumerate(doc_names_sorted_viz3):
    x_center = doc_idx * column_spacing
    
    fig3.add_shape(
        type="rect",
        x0=x_center - column_width/2, x1=x_center + column_width/2,
        y0=0, y1=middle_y_start,
        line=dict(color=doc_colors[doc_name], width=2, dash='dot'),
        fillcolor='rgba(200,200,200,0.05)',
        layer='below'
    )
    
    fig3.add_shape(
        type="rect",
        x0=x_center - column_width/2, x1=x_center + column_width/2,
        y0=middle_y_start, y1=middle_y_end,
        line=dict(color=doc_colors[doc_name], width=3),
        fillcolor='rgba(255,255,255,0)',
        layer='below'
    )
    
    fig3.add_shape(
        type="rect",
        x0=x_center - column_width/2, x1=x_center + column_width/2,
        y0=sentence_y_start, y1=sentence_y_end,
        line=dict(color=doc_colors[doc_name], width=3),
        fillcolor='rgba(200,255,200,0.1)',
        layer='below'
    )
    
    fig3.add_annotation(
        x=x_center, y=-max_chunks * 0.05,
        text=f"<b>{doc_name}</b>",
        showarrow=False,
        font=dict(size=12, color='black'),
        bgcolor='rgba(255,255,255,0.9)',
        borderpad=4
    )
    
    # Add section labels INSIDE each column
    # Knowledge Graph label - positioned inside at the top
    fig3.add_annotation(
        x=x_center,
        y=middle_y_start + 1.5,  # Position inside, just below the top border
        text="<b>Knowledge Graph</b>",
        showarrow=False,
        font=dict(size=11, color='black'),
        xanchor='center',
        yanchor='top'
    )
    
    # Sentences label
    fig3.add_annotation(
        x=x_center,
        y=sentence_y_start + (sentence_y_end - sentence_y_start) * 0.05,
        text="<b>Sentences</b>",
        showarrow=False,
        font=dict(size=11, color='black'),
        xanchor='center',
        yanchor='top'
    )

# Layout
fig3.update_layout(
    title=f"Knowledge Graph - Stage 3: Complete Hierarchy<br>" +
          f"<i>Document Themes (top) → Chunk Graph (middle) → Sentence Children (bottom) | Theme Similarity: {theme_similarity:.3f}</i>",
    xaxis=dict(
        showgrid=False, zeroline=False, showticklabels=False,
        range=[-column_width*0.6, column_spacing + column_width*0.6]
    ),
    yaxis=dict(
        showgrid=False, zeroline=False, showticklabels=False,
        autorange="reversed",
        range=[-max_chunks * 0.08, sentence_y_end * 1.02]
    ),
    plot_bgcolor='rgba(250,250,250,1)',
    height=1100,
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig3.show()

print(f"\n✅ Stage 3 visualization complete!")
print(f"   💡 TOP: Document themes with embedded text")
print(f"   💡 MIDDLE: 2D graph layout with chunk nodes")
print(f"   💡 BOTTOM: Chunks 5, 6, 7 with sentence children")
print(f"   💡 RED LINE: Theme similarity connection ({theme_similarity:.4f})")

# 4.1: Knowledge In 3D
Now let's visualize the knowledge graph structure in 3D space! We'll use PCA to reduce the high-dimensional embeddings down to 3D, then plot all chunks as nodes and their connections as edges.

In [ ]:
#### import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Get all chunk embeddings
print(f"🎨 Creating 3D visualization of knowledge graph")
print(f"   📊 Total chunks: {len(chunk_embeddings)}")

# Extract embeddings and metadata
all_embeddings = []
chunk_ids = []
chunk_docs = []
chunk_texts = []

for chunk_emb in chunk_embeddings:
    all_embeddings.append(chunk_emb.embedding)
    chunk_ids.append(chunk_emb.chunk_id)
    chunk_docs.append(chunk_emb.source_article)
    # Get text preview (first 100 chars)
    text = chunk_emb.chunk_text if hasattr(chunk_emb, 'chunk_text') else ''
    chunk_texts.append(text[:100] + '...' if len(text) > 100 else text)

# Convert to numpy array
embeddings_array = np.array(all_embeddings)
print(f"   🧮 Embedding matrix: {embeddings_array.shape}")

# Step 2: Apply PCA to reduce to 3D
print(f"   🔬 Applying PCA dimensionality reduction...")
pca = PCA(n_components=3, random_state=42)
coords_3d = pca.fit_transform(embeddings_array)

# Scale coordinates to spread out the visualization
scale_factor = 6.0  # Increase this to spread points further apart
coords_3d = coords_3d * scale_factor

explained_var = pca.explained_variance_ratio_
print(f"   ✅ PCA complete - explained variance: {explained_var.sum():.1%}")
print(f"   📏 Applied {scale_factor}x scaling for better spacing")

# Step 3: Create color mapping for documents
unique_docs = list(set(chunk_docs))
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan']
doc_color_map = {doc: colors[i % len(colors)] for i, doc in enumerate(unique_docs)}

print(f"   📚 Documents: {unique_docs}")

# Step 4: Compute FULL similarity matrix and extract strong connections
print(f"   🔗 Computing full similarity matrix for visualization...")
sim_matrix = cosine_similarity(embeddings_array)

# Extract connections above threshold (for visualization only - not sparse!)
threshold = 0.75  # Only visualize strong connections to avoid clutter
edge_pairs = []
for i in range(len(chunk_embeddings)):
    for j in range(i+1, len(chunk_embeddings)):  # Only upper triangle
        if sim_matrix[i, j] > threshold:
            edge_pairs.append((i, j))

print(f"   ✅ Found {len(edge_pairs)} connections above {threshold} threshold")
print(f"   💡 (Using threshold for visualization clarity - full matrix computed)")

# Step 5: Create Plotly 3D scatter plot
fig = go.Figure()

# Add edges (connections between chunks)
edge_x, edge_y, edge_z = [], [], []
for src, dst in edge_pairs:
    edge_x.extend([coords_3d[src, 0], coords_3d[dst, 0], None])
    edge_y.extend([coords_3d[src, 1], coords_3d[dst, 1], None])
    edge_z.extend([coords_3d[src, 2], coords_3d[dst, 2], None])

fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='black', width=1),
    hoverinfo='none',
    showlegend=False,
    name='Connections'
))

# Add nodes (chunks) - one trace per document for colored legend
for doc in unique_docs:
    doc_mask = [d == doc for d in chunk_docs]
    doc_coords = coords_3d[doc_mask]
    doc_texts = [chunk_texts[i] for i, mask in enumerate(doc_mask) if mask]
    
    fig.add_trace(go.Scatter3d(
        x=doc_coords[:, 0],
        y=doc_coords[:, 1],
        z=doc_coords[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color=doc_color_map[doc],
            line=dict(color='black', width=0.5)
        ),
        text=doc_texts,
        hovertemplate='<b>%{text}</b><extra></extra>',
        name=doc
    ))

# Update layout
fig.update_layout(
    title=f"Knowledge Graph 3D Visualization<br>" +
          f"<i>{len(chunk_embeddings)} chunks across {len(unique_docs)} documents | " +
          f"{len(edge_pairs)} connections (>{threshold} similarity) | PCA variance: {explained_var.sum():.1%}</i>",
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3",
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
        bgcolor='rgba(240,240,240,0.9)'
    ),
    height=800,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

print(f"\n✅ 3D visualization created!")
print(f"   💡 Hover over nodes to see chunk text")
print(f"   💡 Drag to rotate, scroll to zoom")
print(f"   💡 Adjust 'scale_factor' and 'threshold' variables to customize")
print(f"\n")

fig.show()

# 4.0 Traversing The Knowledge Graph

Have you ever gone on a "Wikipedia journey" before? Where you first start reading about one topic out of curiosity, then click through similar topics to learn more? This is effectively the same rationale used for traversing knowledge graphs. Rather than attempting to immediately retrieve all relevant information from the "top down", we try to traverse *through* the graph with the goal of extracting the most similar information to what we've already found to answer our questions.

If we were going to do this *algorithmically*, it would look something like this:

1. A user asks an LLM a question.
2. We find some kind of *starting point* within our knowledge graph that is the most relevant to the user's question.
3. From that starting point, we traverse the knowledge graph, extracting semantically relevant information to the user's question.
4. We have some kind of *early stopping* method to prevent extracting too much information.
5. We return the text to the LLM to answer the user's question.

This is *precisely* the objective of the algorithms in this notebook. Additionally, here are some ground rules for our algorithms:

1. They need to be *fast and efficient.* Taking up too much memory or time can bloat a RAG system unnecessarily.
2. They need to be *as accurate and precise as possible* for the use case.
3. They should not provide *too little or too much context* as stated before.
4. Ideally, they should be *AS effective, if not MORE effective, than basic RAG.*

# 5.0: Traversal Algorithms

There are **seven** total algorithms in this repository that can be used for retrieval. Each will be explained in its own section:

1. `basic_retrieval`
2. `query_traversal`
3. `kg_traversal`
4. `triangulation_average`
5. `triangulation_geometric_3d`
6. `triangulation_fulldim`
7. `llm-guided-traversal`

## 5.1: `basic_retrieval`

*Basic semantic RAG algorithm. Contains no traversal. Used as a control.*

![BASIC RETRIEVAL](docs/BASIC_RETRIEVAL.png)

First, embed the user's query.

---
$$\vec{q} = \text{embed}(\text{query})$$
---

Lookup the most semantically similar chunks in the knowledge graph directly based on cosine similarity.

---
$$\text{sim}(\vec{q}, \vec{c}_i) = \frac{\vec{q} \cdot \vec{c}_i}{\|\vec{q}\| \|\vec{c}_i\|}$$
---

Continue selecting the cached top chunks until `max_sentences`, stopping early once sentence quality beats the next chunk option (after at least five sentences are gathered).

---
$$\text{stop when } |S| = \text{max\_sentences}\,\, \text{or}\,\, \Big(|S| \geq 5 \land \max_{s \in S_{\text{extracted}}} \text{sim}(\vec{q}, \vec{s}) > \text{sim}(\vec{q}, \vec{c}_{\text{next}})\Big)$$
---


## 5.2: `query_traversal`

*Query-guided graph traversal that always prioritizes similarity to the original query.*

![QUERY TRAVERSAL](docs/QUERY_TRAVERSAL.png)

Embed the user's query and find the ***most similar chunk node in the graph to the query.*** This is called the *anchor chunk.* 

---
$$\vec{q} = \text{embed}(\text{query}), \quad c_0 = \arg\max_{c_i} \text{sim}(\vec{q}, \vec{c}_i)$$
---

Starting from the anchor chunk's top inter-document and top intra-document connections, at each hop, find the next node (chunk or sentence) most similar to the query.

---
$$n_{t+1} = \arg\max_{n \in \text{neighbors}(c_t)} \text{sim}(\vec{q}, \vec{n})$$
---

Extract all sentences from each visited chunk (including the anchor chunk). Continue traversing to chunks with highest query similarity.

---
$$c_{t+1} = \arg\max_{c \in C_{\text{unvisited}}} \text{sim}(\vec{q}, \vec{c})$$
---

Chunks are not revisited; newly extracted sentences are deduplicated against what has already been gathered.

Stop when `max_sentences` reached or, once at least eight sentences have been collected, when the best extracted sentence exceeds the best available chunk (early stopping).

---
$$\max_{s \in S_{\text{extracted}}} \text{sim}(\vec{q}, \vec{s}) > \max_{c \in C_{\text{available}}} \text{sim}(\vec{q}, \vec{c}) \implies \text{stop}$$
---

## 5.3: `kg_traversal`

*Chunk-centric graph traversal that follows local similarity paths (not query similarity), with a focus on greater graph exploration.*

![KG TRAVERSAL](docs/KG_TRAVERSAL.png)

Similar to the `query_traversal` algorithm, start at the anchor chunk.

---
$$c_0 = \arg\max_{c_i} \text{sim}(\vec{q}, \vec{c}_i), \quad S_0 = \text{sentences}(c_0)$$
---

Unlike the `query_traveral` algorithm, starting from the anchor chunk's `top_x` and `top_k` connections, at each hop, find the next chunk ***most similar to the node that we are currently at in the graph.***

---
$$c_{t+1} = \arg\max_{c \in \text{neighbors}(c_t)} \text{sim}(\vec{c}_t, \vec{c})$$
---

To encourage exploration and prevent too much read-through, prevent sentence overlap by skipping chunks containing already-extracted sentences, and traversal only considers chunk nodes (sentence nodes are ignored).

---
$$c \notin C_{\text{candidates}} \text{ if } \text{sentences}(c) \cap S_{\text{extracted}} \neq \emptyset$$
---

Stop when next chunk similarity ≤ previous hop similarity (exploration-potential early stopping), or at `max_sentences`.

---
$$\text{sim}(\vec{c}_t, \vec{c}_{t+1}) \leq \text{sim}(\vec{c}_{t-1}, \vec{c}_t) \implies \text{stop}$$
---

## 5.4: `triangulation_average`

*Averages the graph edge lengths between the query, current chunk, and prospective chunk/sentence nodes at each step. Creates a balanced averaged traversal that considers both the query and prospective chunks.* When a sentence belongs to a different chunk, sentence-to-chunk similarity is approximated from cached scores so the averaged triangle score stays comparable.


![TRIANGULATION AVERAGE](docs/TRIANGULATION_AVERAGE.png)

Embed query and start at anchor chunk.

---
$$\vec{q} = \text{embed}(\text{query}), \quad c_0 = \arg\max_{c_i} \text{sim}(\vec{q}, \vec{c}_i)$$
---

At each traversal step, identify all graph edges between the current chunk, query, and prospective chunks. Consider these edges as triangles.

---
$$\text{avg}(\vec{q}, \vec{c}_t, \vec{c}_{\text{candidate}}) = \frac{\text{sim}(\vec{q}, \vec{c}_t) + \text{sim}(\vec{q}, \vec{c}_{\text{candidate}}) + \text{sim}(\vec{c}_t, \vec{c}_{\text{candidate}})}{3}$$
---



---
$$n_{t+1} = \arg\max_{n \in \text{neighbors}(c_t)} \text{avg}(\vec{q}, \vec{c}_t, \vec{n})$$
---

Stop when best extracted sentence average exceeds best available chunk average (evaluated once at least eight sentences anchor the check), or when we hit `max_sentences`.

---
$$\max_{s \in S} \text{avg}(\vec{q}, \vec{c}_t, \vec{s}) > \max_{c \in C} \text{avg}(\vec{q}, \vec{c}_t, \vec{c})$$
---
When a sentence belongs to a different chunk, sentence-to-chunk similarity is approximated from cached scores so the averaged triangle score stays comparable.


## 5.5: `triangulation_geometric_3d`

*Geometric triangulation of prospective chunk centroids using PCA-reduced 3D embeddings.*

![TRIANGULATION GEOMETRIC](docs/TRIANGULATION_GEOMETRIC.png)

Reduce all embeddings to 3D using PCA.

---
$$\vec{q}_{3D}, \vec{c}_{i,3D} = \text{PCA}_{1024 \to 3}(\vec{q}, \{\vec{c}_i\})$$
---

Similarly to `trangulation_average`, at each traversal step, identify all graph edges between the current chunk, query, and prospective chunks. This time, find the `triangle centroid` of each created triangle.

---
$$\vec{\text{centroid}} = \frac{\vec{q}_{3D} + \vec{c}_{t,3D} + \vec{n}_{3D}}{3}$$
---

Traverse to the node with its centroid closest to query (minimal Euclidean distance).

---
$$n_{t+1} = \arg\min_{n \in \text{neighbors}(c_t)} \|\vec{\text{centroid}}(\vec{q}, \vec{c}_t, \vec{n}) - \vec{q}_{3D}\|_2$$
---


## 5.6: `triangulation_fulldim`

Geometric triangulation in *full* embedding space.

Work directly with full embeddings instead of PCA 3D reduction.

---
$$\vec{q}, \vec{c}_i \in \mathbb{R}^{d}\quad(\text{dimension auto-detected per knowledge graph, default }d=1024)$$
---

Similarly to the other triangulation algorithms, identify triangles. But this time, in full embedding dimensional space.

---
$$\vec{\text{centroid}}_{1024D} = \frac{\vec{q} + \vec{c}_t + \vec{n}}{3}$$
---

Select node with centroid closest to query in full-dimensional Euclidean space.

---
$$n_{t+1} = \arg\min_{n \in \text{neighbors}(c_t)} \|\vec{\text{centroid}}(\vec{q}, \vec{c}_t, \vec{n}) - \vec{q}\|_2$$
---

Most mathematically rigorous approach, preserves all embedding information.

---
$$\text{Edge lengths: } d(\vec{q}, \vec{c}) = \|\vec{q} - \vec{c}\|_2$$
---

## 5.7: `llm-guided-traversal`

Modified version of `query_traversal` but uses a lightweight LLM instead. Trades speed and cost for accuracy.

![LLM GUIDED TRAVERSAL](docs/LLM_GUIDED_TRAVERSAL.png)

Embed query and find anchor chunk (same as other algorithms).

---
$$\vec{q} = \text{embed}(\text{query}), \quad c_0 = \arg\max_{c_i} \text{sim}(\vec{q}, \vec{c}_i)$$
---

At each hop, get `top_k` and `top_x` potential chunks based on query similarity.

---
$$C_{\text{candidates}} = \text{top-k}\{c \in \text{neighbors}(c_t) \mid \text{sim}(\vec{q}, \vec{c})\}$$
---

---
At each hop, the LLM receives this prompt:

```
You are a knowledge graph traversal agent. Your goal: find relevant content to answer the query.

QUERY: {user's question}

ALREADY EXTRACTED ({n} sentences):
{summary of first 5 sentences extracted so far...}

CANDIDATE CHUNKS (pick ONE or STOP):
1. [chunk_id_1] (similarity: 0.85)
   Preview: {first 200 chars of chunk 1}...

2. [chunk_id_2] (similarity: 0.78)
   Preview: {first 200 chars of chunk 2}...

3. [chunk_id_3] (similarity: 0.72)
   Preview: {first 200 chars of chunk 3}...

4. [chunk_id_4] (similarity: 0.69)
   Preview: {first 200 chars of chunk 4}...

5. [chunk_id_5] (similarity: 0.65)
   Preview: {first 200 chars of chunk 5}...

INSTRUCTIONS:
- Choose the chunk number (1-5) that seems most relevant to answering the query
- If you believe we have enough information to answer the query, respond with "stop"
- Consider both what we've already extracted and what new information each candidate provides
- Respond ONLY with a JSON object in this exact format:

{"choice": <number 1-5 OR "stop">, "reasoning": "brief explanation"}

Your response:
```
---

Send the user's query, the currently extracted context, and previews of potential chunks to LLM. LLM chooses next chunk or stops.

If the LLM response is invalid, fall back to the highest query similarity candidate to keep traversal moving.

---
$$c_{t+1} = \text{LLM}(\text{query}, S_{\text{extracted}}, C_{\text{candidates}})$$
---

LLM decides when to stop based on semantic reasoning (not just similarity).

---
$$\text{LLM decides: continue or stop based on context sufficiency}$$
---

# 6. Retrieval Testing

This section lets you test any of the seven algorithms on the cached knowledge graph. Simply choose your algorithm and query, then run the cells to see traversal logs and visualizations.

**Available Algorithms:**
1. `basic_retrieval` - Basic semantic RAG (no traversal, control baseline)
2. `query_traversal` - Query-guided graph traversal
3. `kg_traversal` - Chunk-centric local similarity traversal
4. `triangulation_average` - Averaged triangle score traversal
5. `triangulation_geometric_3d` - Geometric triangulation (3D PCA)
6. `triangulation_fulldim` - Full-dimensional geometric triangulation
7. `llm_guided_traversal` - LLM-guided query traversal

## 6.1: Choose Your Algorithm and Query

In [ ]:
# ============================================================================
# CHOOSE YOUR ALGORITHM AND QUERY
# ============================================================================

# Select one of the seven algorithms
algorithm = "llm_guided_traversal"  # Change this to any algorithm from the list above

# Define your query
query = "What is the psychology behind learning? Do computers learn like humans do?"  # Change this to your question

# ============================================================================
# RETRIEVAL PARAMETERS (Optional - modify to extend or limit retrieval)
# ============================================================================

# Maximum sentences to retrieve (default: 20)
# Increase this to 50-100+ to really fly through the graph and explore more
max_sentences = 50  

# Maximum hops/jumps through the graph (safety limit, default: 10)
max_hops = 20

# Enable early stopping (stops when best sentence beats best available chunk)
enable_early_stopping = False  # Set to False to always retrieve max_sentences

# ============================================================================
# Apply custom parameters by temporarily modifying config
# ============================================================================

# Store original values to restore later
original_max_sentences = config['retrieval']['semantic_traversal'].get('min_sentence_threshold', 20)
original_max_hops = config['retrieval']['semantic_traversal'].get('max_safety_hops', 50)
original_early_stopping = config['retrieval']['semantic_traversal'].get('enable_early_stopping', True)
original_max_results = config['retrieval']['semantic_traversal'].get('max_results', 50)

# Apply custom parameters
config['retrieval']['semantic_traversal']['min_sentence_threshold'] = max_sentences
config['retrieval']['semantic_traversal']['max_safety_hops'] = max_hops
config['retrieval']['semantic_traversal']['enable_early_stopping'] = enable_early_stopping
config['retrieval']['semantic_traversal']['max_results'] = max_sentences  

print(f"🎯 Algorithm: {algorithm}")
print(f"🔍 Query: '{query}'")
print(f"⚙️  Max Sentences: {max_sentences}")
print(f"⚙️  Max Results: {max_sentences}")
print(f"⚙️  Max Hops: {max_hops}")
print(f"⚙️  Early Stopping: {'Enabled' if enable_early_stopping else 'Disabled'}")
print(f"✅ Ready to retrieve!")


## 6.2: Run Retrieval

This will execute the selected algorithm and display traversal logs.

In [ ]:
# Run retrieval using the cached knowledge graph and retrieval orchestrator
print(f"🚀 Running {algorithm} algorithm...")
print("=" * 80)

result = retrieval_orchestrator.retrieve(query, algorithm)

print("=" * 80)
print(f"\n✅ Retrieval complete!")
print(f"   📊 Algorithm: {result.algorithm_name}")
print(f"   📝 Retrieved {len(result.retrieved_content)} sentences")
print(f"   ⭐ Final score: {result.final_score:.3f}")
print(f"   ⏱️  Processing time: {result.processing_time:.3f}s")

# Show preview of retrieved content
print(f"\n📄 Preview of first 3 retrieved sentences:")
for i, sentence in enumerate(result.retrieved_content[:3], 1):
    preview = sentence[:150] + "..." if len(sentence) > 150 else sentence
    print(f"   {i}. {preview}")

# Restore original config values
config['retrieval']['semantic_traversal']['min_sentence_threshold'] = original_max_sentences
config['retrieval']['semantic_traversal']['max_safety_hops'] = original_max_hops
config['retrieval']['semantic_traversal']['enable_early_stopping'] = original_early_stopping
config['retrieval']['semantic_traversal']['max_results'] = original_max_results


## 6.2.1: Generate Answer from Retrieved Content

In [ ]:
# DEBUG: Print ALL retrieved sentences to verify content
print("🔍 DEBUG - Full retrieved content:")
print("=" * 80)
for i, sentence in enumerate(result.retrieved_content, 1):
    print(f"{i}. {sentence}")
    print()
print("=" * 80)


In [ ]:
# ============================================================================
# ANSWER GENERATION CONFIGURATION (Choose your provider and model)
# ============================================================================

# Choose provider: "openai", "openrouter", or "ollama"
answer_provider = "ollama"

# Choose model name based on provider:
# - OpenAI: "gpt-4o", "gpt-4o-mini", "gpt-3.5-turbo"
# - OpenRouter: "meta-llama/llama-3.3-70b-instruct", "anthropic/claude-3.5-sonnet", etc.
# - Ollama: "llama3", "mistral", "phi", etc.
answer_model_name = "llama3.2"

# Temperature (0.0 = deterministic, 1.0 = creative)
answer_temperature = 0.1

# Max tokens for answer
answer_max_tokens = 1000

# ============================================================================
# Generate answer using the retrieved content
# ============================================================================
from evaluation.models import ModelManager

print("🤖 Generating answer from retrieved content...")
print("=" * 80)

try:
    # Store original settings (use .get() to handle missing keys)
    answer_config = config.get('deepeval', {}).get('models', {}).get('answer_generation', {})
    original_provider = answer_config.get('provider', 'openai')
    original_model = answer_config.get('model_name', 'gpt-4o')
    original_temp = answer_config.get('temperature', 0.1)
    original_tokens = answer_config.get('max_tokens', 2000)
    
    # Apply custom settings
    if 'deepeval' not in config:
        config['deepeval'] = {}
    if 'models' not in config['deepeval']:
        config['deepeval']['models'] = {}
    if 'answer_generation' not in config['deepeval']['models']:
        config['deepeval']['models']['answer_generation'] = {}
    
    config['deepeval']['models']['answer_generation']['provider'] = answer_provider
    config['deepeval']['models']['answer_generation']['model_name'] = answer_model_name
    config['deepeval']['models']['answer_generation']['temperature'] = answer_temperature
    config['deepeval']['models']['answer_generation']['max_tokens'] = answer_max_tokens
    
    # Initialize model manager with custom settings
    model_manager = ModelManager(config, logger)
    answer_model = model_manager.get_answer_generation_model()
    
    # Show which provider is being used
    print(f"📡 Using: {answer_provider} / {answer_model_name}")
    
    # Format retrieved sentences as context
    context = "\n".join(result.retrieved_content)
    
    print(f"📊 Context: {len(result.retrieved_content)} sentences, {len(context)} characters")
    
    # Create prompt for answer generation
    prompt = f"""Based on the provided context, answer the following question. Use only the information from the context and be concise and accurate.

Context:
{context}

Question: {query}

Answer:"""
    
    # Generate the answer
    print(f"⏳ Generating response...")
    response = answer_model.generate(prompt)
    
    # Extract answer text - handle multiple response formats
    if isinstance(response, tuple):
        # Ollama sometimes returns (text, score) tuples
        answer = response[0] if len(response) > 0 else str(response)
    elif hasattr(response, 'response'):
        answer = response.response
    elif isinstance(response, str):
        answer = response
    else:
        answer = str(response)
    
    print(f"\n💬 Generated Answer:\n")
    print(f"{answer}")
    print("\n" + "=" * 80)
    
    # Restore original config
    config['deepeval']['models']['answer_generation']['provider'] = original_provider
    config['deepeval']['models']['answer_generation']['model_name'] = original_model
    config['deepeval']['models']['answer_generation']['temperature'] = original_temp
    config['deepeval']['models']['answer_generation']['max_tokens'] = original_tokens
    
except Exception as e:
    import traceback
    print(f"\n⚠️  Answer generation failed: {e}")
    print(f"\n🔍 Full error traceback:")
    traceback.print_exc()
    print(f"\n💡 Tip: Check your provider settings and API keys:")
    print(f"   - OpenAI: Set OPENAI_API_KEY environment variable")
    print(f"   - OpenRouter: Set OPENROUTER_API_KEY environment variable")
    print(f"   - Ollama: Ensure Ollama server is running (ollama serve)")
    print("\n" + "=" * 80)
    
    # Restore original config even on error
    try:
        config['deepeval']['models']['answer_generation']['provider'] = original_provider
        config['deepeval']['models']['answer_generation']['model_name'] = original_model
        config['deepeval']['models']['answer_generation']['temperature'] = original_temp
        config['deepeval']['models']['answer_generation']['max_tokens'] = original_tokens
    except:
        pass


## 6.3: Visualize Traversal Path

Generate 2D and 3D visualizations of the retrieval path.

In [ ]:
# Import visualization functions
from utils.matplotlib_visualizer import create_heatmap_visualization
from utils.plotly_visualizer import create_algorithm_visualization
import matplotlib.pyplot as plt

print("✅ Visualization functions imported")

In [ ]:
# 2D Global View - Shows complete document landscape with full traversal path
print("🎨 Creating 2D GLOBAL visualization (strategic overview)...")

fig_matplotlib_global = create_heatmap_visualization(
    result=result,
    query=query,
    knowledge_graph=knowledge_graph,
    figure_size=(20, 8),
    max_documents=6,
    visualization_type="global"
)

plt.tight_layout()
plt.show()

print("✅ Global visualization displayed!")


In [ ]:
# 2D Sequential View - Shows hop-by-hop traversal analysis
print("🎨 Creating 2D SEQUENTIAL visualization (hop-by-hop analysis)...")

fig_matplotlib_sequential = create_heatmap_visualization(
    result=result,
    query=query,
    knowledge_graph=knowledge_graph,
    figure_size=(20, 8),
    visualization_type="sequential"
)

plt.tight_layout()
plt.show()

print("✅ Sequential visualization displayed!")
print("💡 Look for the step numbers on each marker to see the traversal order!")


In [ ]:
# 3D Interactive View - Plotly visualization with rotation and zoom
print("🎨 Creating 3D INTERACTIVE visualization...")

fig_plotly = create_algorithm_visualization(
    result=result,
    query=query,
    knowledge_graph=knowledge_graph,
    method="pca",
    max_nodes=200,  # Increased from 50 to show all traversal nodes
    show_all_visited=True,
    edge_threshold=0.70  # 0.6=very dense, 0.75=moderate, 0.8=sparse, 0.85=very sparse
)

fig_plotly.show()

# Export to temp_data directory
import os
from datetime import datetime
os.makedirs("temp_data", exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_path = f"temp_data/knowledge_graph_visualization_{timestamp}.html"
fig_plotly.write_html(export_path)
print(f"💾 Visualization exported to: {export_path}")

print("✅ Plotly 3D visualization displayed!")
print("💡 Tip: Drag to rotate, scroll to zoom, and hover over nodes for details!")

# 7. Benchmarking

Benchmarking RAG systems can be done via a variety of different tools and libraries but it is primarily done via a series of universal metrics:

- Precision: How relevant the retrieved chunks are for answering the question.
- Recall: Whether all of the necessary info was retrieved to answer the question.
- Faithfulness: How grounded in fact the generated answer and retrieved contexts are.
- Answer Relevance: How relevant the geenerated answer was at actually answering the question.

These metrics are tracked using libraries like `ragas` and `deepeval`. For this project, I used `deepeval` due to the modular dataset generation and context grouping support.

## 7.1 DeepEval

The full benchmarking pipeline with `deepeval` can be viewed in **2 stages.**

1. Synthetic Dataset Generation:
   - Using our knowledge graph, create **context groups** of similar contexts that we send to `deepeval`'s question generation engine.
   - Deepeval then uses a chosen LLM to generate a question from these contexts, as well as an expected answer for groundedness.
   - We can then use `deepeval`'s *evolution* feature to evolve the question to be slightly more complex (reasoning, multi-context, etc).
   - Finally, we can export the dataset as a `.csv` to be pushed to the dashboard, or cached locally.
2. Full Evaluation:
   - For each algorithm selected, we run through the full dataset and embed each generated query and attempt to retrieve the same contexts used for that question. We also attempt to generate an accurate answer. Each step in this process is done using any LLM of our choosing.
   - `deepeval` then takes our responses and sends them to a critic LLM for an analysis on the metrics listed previously.
   - We can then see the results in the `deepeval` dashboard, with detailed JSON responses from the critic model.

In the next sections, we'll walk through this process in detail to demonstrate how each dataset was generated, and how you can modify the `config.yaml` file to attempt retrieval on your own if you so choose.

## 7.2 Context Grouping Algorithms

There are a variety of different available options online for how to do context grouping. For this project, I wanted to see if I could, effectively, create context grouping algorithms that functioned extremely similarly to our retrieval algorithms by traversing the knowledge graph. There are currently *four* methods of context grouping available in this repository. Only *one* was used to generate the final dataset, but the other four are still available for use. The context grouping parameters can be customized using the `config.yaml`.

---
### `intra_document`

Starting at a random chunk, traverse to the most similar chunk to the current chunk in the *current document* that ***does not contain a sentence we've already seen.*** This allows for a little bit more exploration. 

```
context_strategies:
  intra_document:
    enabled: true
    weight: 0.33
    max_sentences: 10
    description: "Within-document exploratory context grouping."
```

---
### `theme_based` (used for the 20 question dataset)

Starting at a random chunk, rank other documents by theme similarity. Then take a look at the list of connected chunks from different documents that contain the highest theme overlap by theme similarity. Then at each step, traverse **between documents** to the most similar chunk to the current chunk. This creates an "oscilation" between multiple documents of similar themes.

```
context_strategies:
  theme_based:
    enabled: true
    weight: 0.33
    max_sentences: 10
    fallback_to_inter_document: true
    description: "Cross-document thematic context grouping."
```

---
### `sequential_multi_hop` (used for the 50 question dataset)

Starting at a random chunk, "read" forwards or backwards inside the current document (whichever is most similar). Then after reaching 5 sentences, do a `theme_based` hop to the most similar chunk in a different document. Then, "read" forwards or backwards again. We do this so that we don't **just** hop between documents during retrieval; we actually want to attempt to get our algorithms to "read" a little bit during retrieval, and this context grouping algorithm is designed to demonstrate that once we attempt retrieval.

```
context_strategies:
  sequential_multi_hop:
    enabled: true
    weight: 0.33
    num_reading_hops: 3
    num_paragraph_sentences: 5
    num_cross_doc_hops: 3
    description: "Cross-document thematic reading simulation: 3 docs × 5 sentences = 15-sentence narratives"
```

---
### `deepeval_native`

Basic context grouping. Uses a few random or sequential chunks, basic sentence deduplication, and `deepeval`'s quality filtration. Useable as a baseline but isn't very robust.

```
context_strategies:
  deepeval_native:
    enabled: true
    weight: 0.2
    max_sentences: 10
    extraction_mode: "random"
    chunks_per_group: 3
    ensure_document_diversity: true
    description: "Simple random/sequential extraction with DeepEval FiltrationConfig quality filtering - no semantic traversal"
```

---




## 7.3 Dataset Generation: Goldens, Filtration, and Evolutions

Here is the relevant section in the `config.yaml`:

```
deepeval:
  dataset:
    generation:
      num_goldens: 50
      max_goldens_per_context: 1
      include_expected_output: true

    filtration:
      enabled: true  # DeepEval FiltrationConfig only supports OpenAI models, not Ollama
      critic_model: "gpt-4o-mini"  # Only used if enabled=true
      synthetic_input_quality_threshold: 0.8
      max_quality_retries: 5

    evolution:
      enabled: true
      num_evolutions: 1
      evolution_types:
        - "REASONING"
        - "COMPARATIVE"
        - "MULTICONTEXT"
      evolution_distribution:
        REASONING: 0.4
        COMPARATIVE: 0.2
        MULTICONTEXT: 0.4

    output:
      save_path: "data/synthetic_dataset.json"
      cache_enabled: true
      force_regenerate: true
      push_to_dashboard: false
      dataset_alias: "50qa-seq-multihop-gpt4o-reasoning-comparative-multicontext"
      pull_from_dashboard: true
      generate_csv: true
      csv_context_delimiter: " | "
```

### Relevant Terms:

- `num_goldens`: A *golden* is a question/answer/context entry in a synthetic dataset generated by DeepEval. We can choose how many we want to generate based on our goals. For this research, we used 50.
- `include_expected_output`: This ensures we actually generate an expected answer in the dataset to compare against during evaluation.
- `filtration -> enabled: true` *Filtration* takes each generated golden and sends it to the `critic_model` to be analyzed for quality. If the question/answer/context grouping is below the `synthetic_input_quality_threshold`, we retry up to the `max_quality_retries`.
- `evolution`: *Evolution*, unique to DeepEval, takes the generated question and re-generates it according to a specific `evolution_type`. They could be mixed and matched but for our 50 question dataset in this research, we only used 1 evolution per golden, weighted at the `evolution_distribution` above.
- `output`: These settings allow configuration for synthetic dataset generation. You can either pull a previously built dataset from the DeepEval dashboard, or you can push one to it directly (you have to pay for DeepEval starter plan to push). By default, the dataset is saved as a `.json` but you can optionally export a `.csv` to the same directory to upload to DeepEval if you don't want to pay for it.
- `dataset_alias`: The name of the dataset.

## 7.4 Synthetic Datasets

This repository contains *three* datasets that have been pre-generated for convenience:

- `1q-intradoc-reasoning-multicontext`: Single question for testing. Uses intradoc context grouping, with a dual reasoning/multicontext evolution question.
- `20q-themes-gpt4omini-reasoning`: 20 questions for testing. Uses theme context grouping, with `gpt-4o-mini` question generation and filtration, with only reasoning evolutions.
- `50qa-seq-multihop-gpt4o-reasoning-comparative-multicontext`: Full 50 question dataset. Uses sequential multihop context grouping, `gpt-4o` for question generation and filteration, and using 40% reasoning, 40% multicontext, and 20% comparative evolutions.

You may generate your own if you wish for your own testing.

## 7.5 Models

There are multiple different model configs in the `config.yaml`, since different models are used at different parts of the benchmarking process:

```
models:
  embedding_models:
    - "mixedbread-ai/mxbai-embed-large-v1"

dataset:
  filtration:
    enabled: true  # DeepEval FiltrationConfig only supports OpenAI models, not Ollama
    critic_model: "gpt-4o-mini"  # Only used if enabled=true
    synthetic_input_quality_threshold: 0.8
    max_quality_retries: 5

deepeval:
  models:
    question_generation: 
      provider: "openai"  
      model_name: "gpt-4o"  
      temperature: 0.1
      max_tokens: 20000

    answer_generation:
      provider: "openai" 
      model_name: "gpt-5-nano"  
      temperature: 0.1
      max_tokens: 5000

    evaluation_judge:
      provider: "openai"  
      model_name: "gpt-5-nano" 
      temperature: 0.0
      max_tokens: 50000


```

### Relevant Terms:

- `embedding_models:`: Embedding model for entire project. Strongly recommend keeping as is, this model is 1024 dimensions and is very effective.
- `provider:` Can choose any from `"openai"`, `"openrouter"`, or `"ollama"`. Strongly recommend `"openai"` due to issues with `openrouter` during long evaluation sessions but the options are available to anyone looking to experiment.
- `critic_model`: Critic model for dataset filtration. This model identifies if a generated golden is sufficient, if not we retry.
- `question_generation`: Model for actually generating questions through evolutions. Recommend keeping this a high-quality model.
- `answer_generation`: Model for generating answers during evaluation, as well as generating the `expected_output` in a golden (expected answer).
- `evaluation_judge`: Model evaluates the precision, recall, relevancy, and faithfulness of each golden, returning scores and reasoning for each. This gets *very expensive*, as each golden requires multiple API calls with large input token volume. `gpt-5-nano` is a good sweet spot for cost to performance.

# 8.0 Evaluation

Running a full DeepEval evaluation in this notebook would be unreasonable. But the resources to do so are available in the README for this project. If you want to run a full evaluation from scratch using the resources in this notebook, carefully modify the `config.yaml` and then run the `benchmark.py` script. It will take care of everything for you.

For now, let's take a look at the results for all of the algorithms, with the relevant parameters.

### Global Parameters:

- `embed_model`: "mixedbread-ai/mxbai-embed-large-v1"
- `reranking`: False
- `models -> provider`: openai
- `models -> question_generation`: gpt-4o
- `models -> answer_generation`: gpt-5-nano
- `models -> evaluation_judge`: gpt-5-nano
- `chunking -> strategy`: "sliding_window"
- `chunking -> window_size`: 3
- `chunking -> overlap`: 2
- `wikipedia -> topics`: "Machine Learning, Artificial Intelligence"
- `wikipedia -> articles_per_topic`: 5
- `wikipedia -> max_article_length`: 5000000

### 20 Question Dataset:

```
deepeval:
  models:
    question_generation: 
      provider: "openai"  
      model_name: "gpt-4o-mini"  
      temperature: 0.1
      max_tokens: 20000

    answer_generation:
      provider: "openai"  
      model_name: "gpt-4o-mini" 
      temperature: 0.1
      max_tokens: 5000

    evaluation_judge:
      provider: "openai"  
      model_name: "gpt-4o-mini"  
      temperature: 0.0
      max_tokens: 50000

  dataset:
    generation:
      num_goldens: 20
      max_goldens_per_context: 1
      include_expected_output: true

    filtration:
      enabled: true  
      critic_model: "gpt-4o-mini"  
      synthetic_input_quality_threshold: 0.8
      max_quality_retries: 5

    evolution:
      enabled: true
      num_evolutions: 1
      evolution_types:
        - "REASONING"
        # - "COMPARATIVE"
        # - "MULTICONTEXT"
      evolution_distribution:
        REASONING: 1.0
        COMPARATIVE: 0.0
        MULTICONTEXT: 0.0

    output:
      save_path: "data/synthetic_dataset.json"
      cache_enabled: true
      force_regenerate: true
      push_to_dashboard: false
      dataset_alias: "20qa-themes-gpt4omini-reasoning"
      pull_from_dashboard: false
      generate_csv: true
      csv_context_delimiter: " | "

context_strategies:
  theme_based:
    enabled: false
    weight: 0.33
    max_sentences: 10
    fallback_to_inter_document: true
    description: "Cross-document theme overlap traversal with semantic fallback"
```


### 50 Question Dataset:

```
deepeval:
  models:
    question_generation:
      provider: "openai"  
      model_name: "gpt-4o" 
      temperature: 0.1
      max_tokens: 20000

    answer_generation:
      provider: "openai"
      model_name: "gpt-5-nano" 
      temperature: 0.1
      max_tokens: 5000

    evaluation_judge:
      provider: "openai"  
      model_name: "gpt-5-nano" 
      temperature: 0.0
      max_tokens: 50000

  dataset:
    generation:
      num_goldens: 50
      max_goldens_per_context: 1
      include_expected_output: true

    filtration:
      enabled: true  
      critic_model: "gpt-4o-mini" 
      synthetic_input_quality_threshold: 0.8
      max_quality_retries: 5

    evolution:
      enabled: true
      num_evolutions: 1
      evolution_types:
        - "REASONING"
        - "COMPARATIVE"
        - "MULTICONTEXT"
      evolution_distribution:
        REASONING: 0.4
        COMPARATIVE: 0.2
        MULTICONTEXT: 0.4

    output:
      save_path: "data/synthetic_dataset.json"
      cache_enabled: true
      force_regenerate: true
      push_to_dashboard: false
      dataset_alias: "50qa-seq-multihop-gpt4o-reasoning-comparative-multicontext"
      pull_from_dashboard: true
      generate_csv: true
      csv_context_delimiter: " | "

context_strategies:
  sequential_multi_hop:
    enabled: true
    weight: 0.33
    num_reading_hops: 3
    num_paragraph_sentences: 5
    num_cross_doc_hops: 3
    description: "Structured reading simulation: 3 docs × 5 sentences = 15-sentence narratives"
```


---
## $$\text{20qa-themes-gpt4omini-reasoning}$$

$$
\begin{array}{|l|c|c|c|c|c|}
\hline
\textbf{Algorithm} & \textbf{Precision} & \textbf{Recall} & \textbf{Answer Relevancy} & \textbf{Faithfulness} & \textbf{Test Cases} \\
\hline
\text{basic\_retrieval} & 0.87 & 0.74 & 0.91 & 0.93 & 16/20 \\
\hline
\text{query\_traversal} & 0.83 & 0.83 & 0.91 & 1.00 & 16/20 \\
\hline
\text{kg\_traversal} & 0.73 & 0.72 & 0.98 & 0.92 & 15/20 \\
\hline
\text{triangulation\_average} & 0.84 & 0.77 & 0.96 & 1.00 & 16/20 \\
\hline
\text{triangulation\_geometric\_3d} & 0.86 & 0.77 & 0.96 & 1.00 & 16/20 \\
\hline
\text{triangulation\_fulldim} & 0.90 & 0.78 & 0.95 & 0.99 & 17/20 \\
\hline
\text{llm\_guided\_traversal} & 0.88 & 0.82 & 0.95 & 1.00 & 17/20 \\
\hline
\end{array}
$$


---
## $$\text{50qa-seq-multihop-gpt4o-reasoning-comparative-multicontext}$$

$$
\begin{array}{|l|c|c|c|c|c|}
\hline
\textbf{Algorithm} & \textbf{Precision} & \textbf{Recall} & \textbf{Answer Relevancy} & \textbf{Faithfulness} & \textbf{Test Cases} \\
\hline
\text{basic\_retrieval} & 0.93 & 0.88 & 0.99 & 0.99 & 48/50 \\
\hline
\text{query\_traversal} & 0.91 & 0.91 & 0.98 & 1.00 & 50/50 \\
\hline
\text{kg\_traversal} & 0.93 & 0.87 & 0.99 & 0.99 & 49/50 \\
\hline
\text{triangulation\_average} & 0.92 & 0.87 & 0.98 & 0.99 & 49/50 \\
\hline
\text{triangulation\_geometric\_3d} & 0.93 & 0.85 & 0.98 & 1.00 & 48/50 \\
\hline
\text{triangulation\_fulldim} & 0.93 & 0.87 & 1.00 & 0.97 & 47/50 \\
\hline
\text{llm\_guided\_traversal} & 0.91 & 0.94 & 0.99 & 0.99 & 49/50 \\
\hline
\end{array}
$$